# Import and Load Data

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
import itertools
import tensorflow as tf
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv('./sample_data/amazon_reviews_us_Watches_v1_00.tsv',sep='\t',engine='python',on_bad_lines=lambda x:x[:-1])

In [ ]:
data.shape

(959868, 15)

In [ ]:
data = data[:450000]

In [ ]:
data.shape

(450000, 15)

# Preproccess

In [ ]:
# Drop the null examples
data = data.dropna()

In [ ]:
# We just need Star_Rating and Review_body column
data = data[["star_rating","review_body"]]

In [ ]:
# We should'nt proccess 3 star products because they're nutural
data = data[data['star_rating']!=3]

In [ ]:
data.shape

(414542, 2)

Linguistic Changes

In [ ]:

remove_non_alphabet = lambda x: re.sub('[^a-zA-Z]',' ',str(x))
remove_url = lambda x:re.sub(r'http\S+','',x)
data['review_body'] = data['review_body'].map(remove_non_alphabet)
data['review_body'] = data['review_body'].map(remove_url)
data['review_body'] = data['review_body'].str.lower()

In [ ]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"wouldn't", "would not",s)
    s = re.sub(r"couldn't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s = re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"'’m", " am", s)
    return s
data['review_body']=data['review_body'].apply(lambda x:contractions(x))
data['review_body']=data['review_body'].apply(lambda x:re.sub(r' +',' ',x))
stop = stopwords.words('english')
data['review_body']=data['review_body'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

Labeling

In [ ]:
for index,row in data['star_rating'].items():
    if data['star_rating'][index]>=4:
        data.loc[index,'label'] = 1
    elif data['star_rating'][index]<=2:
        data.loc[index,'label'] = 0
data = data[['review_body','label']]

Train Test Split

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(data['review_body'], data['label'], test_size=0.2)
vectorizer= TfidfVectorizer(ngram_range = (1,3), max_features = 1000)
tf_x_train = vectorizer.fit_transform(x_train)
tf_x_test = vectorizer.fit_transform(x_test)
# Now we have vectorized X_train and X_test

# Machine learning Algorithms(Naive Bays and Neural Network)

---



naive Bays

In [ ]:
nb_model = GaussianNB()
nb_model.fit(tf_x_train.toarray(),y_train)
y_pred = nb_model.predict(tf_x_test.toarray())

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.23      0.45      0.31     12799
         1.0       0.88      0.73      0.80     70110

    accuracy                           0.69     82909
   macro avg       0.56      0.59      0.55     82909
weighted avg       0.78      0.69      0.72     82909



Neural Networks

In [ ]:
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation='relu',input_shape=(1000,)),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
nn_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy',metrics=['accuracy'])

history = nn_model.fit(
    tf_x_train.toarray(), y_train, epochs=100, batch_size=32, validation_split=0.25, verbose=0
)

y_pred = nn_model.predict(tf_x_test.toarray())
y_pred = (y_pred > 0.5).astype(int).reshape(-1,)
# y_pred

print(classification_report(y_test,y_pred))

2591/2591 [==============================] - 4s 2ms/step
              precision    recall  f1-score   support

         0.0       0.21      0.26      0.23     12799
         1.0       0.86      0.82      0.84     70110

    accuracy                           0.74     82909
   macro avg       0.54      0.54      0.54     82909
weighted avg       0.76      0.74      0.75     82909

